In [49]:
import csv
import pandas as pd
import numpy as np
import re

In [69]:
skills = pd.read_csv('skills.csv')
texts = pd.read_csv('temp_specs.csv')

In [70]:
texts = texts.drop(['index', 'YEAR', 'WAGE_OFFER_FROM', 'WAGE_OFFER_TO', 'WAGE_OFFER_UNIT_OF_PAY', 'WORKSITE_CITY', 'WORKSITE_STATE'], 1)
texts = texts.drop_duplicates(ignore_index=True)
texts

<ipython-input-70-a7c46a0a555b>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  texts = texts.drop(['index', 'YEAR', 'WAGE_OFFER_FROM', 'WAGE_OFFER_TO', 'WAGE_OFFER_UNIT_OF_PAY', 'WORKSITE_CITY', 'WORKSITE_STATE'], 1)


,EMPLOYER_NAME,JOB_TITLE,MINIMUM_EDUCATION,SPECIFIC_SKILLS
0,400 CAPITAL MANAGEMENT LLC,ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH,Master's,NaN
1,ABN AMRO CLEARING CHICAGO LLC,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST,Bachelor's,Bachelors degree or foreign equivalent plus te...
2,ACADIAN ASSET MANAGEMENT LLC,DEVOPS ENGINEER,Master's,"Continued from H. 10B. Similar experience, inc..."
3,ACADIAN ASSET MANAGEMENT LLC,SENIOR BUSINESS ANALYST,Bachelor's,Must have seven 7 years of experience in each ...
4,ACADIAN ASSET MANAGEMENT LLC,SENIOR SOFTWARE ENGINEER,Bachelor's,Must have five 5 years of experience in the fo...
...,...,...,...,...
5848,WELLS FARGO BANK NA,APPLICATION SYSTEMS ENGINEER 5,Bachelor's,MUST HAVE BACHELORS DEGREE IN COMPUTER SCIENCE...
5849,WELLS FARGO BANK NA,SOFTWARE ENGINEER SR.,Bachelor's,Requires a Bachelors degree in Electronic Engi...
5850,WILLIAM BLAIR COMPANY LLC,VICE PRESIDENT,Bachelor's,Education Requirement Bachelors degree or fore...
5851,WORLDQUANT LLC,ASSOCIATE QUANTITATIVE RESEARCH & OPTIMIZATION,Master's,"Background in education, training or experienc..."


In [71]:
roles = pd.read_csv("roles.csv", low_memory=False)
company_roles = pd.read_csv("company_roles.csv", low_memory=False)
companies = pd.read_csv("companies.csv", low_memory=False)
companies2 = pd.read_csv("companies_NSCC-MPID_fuzzymatched.csv", low_memory=False)
companies3 = pd.read_csv("companies_H1B_fuzzymatched.csv", low_memory=False)
companies = companies.append(companies2, ignore_index = True)
companies = companies.append(companies3, ignore_index = True)
companies = companies[['COMPANY_ID', 'NAME']]
companies

,COMPANY_ID,NAME
0,0,400 CAPITAL MANAGEMENT LLC
1,1,ABN AMRO CLEARING CHICAGO LLC
2,2,ACHELOUS PARTNERS
3,3,AES US SERVICES LLC
4,4,AKUNA CAPITAL LLC
...,...,...
552,552,GATEWOOD WEALTH SOLUTIONS
553,553,B RILEY FBR INC
554,554,ALTISOURCE SOLUTIONS INC
555,555,PEOPLE'S UNITED ADVISORS INC


In [72]:
company_roles = company_roles.merge(companies, on='COMPANY_ID', how='left')
company_roles = company_roles.merge(roles, on='ROLE_ID', how='left')
company_roles = company_roles.rename(columns={'NAME_x': 'COMPANY_NAME', 'NAME_y': 'ROLE_NAME'})
company_roles

,COMPANY_ROLES_ID,COMPANY_ID,ROLE_ID,COMPANY_NAME,ROLE_NAME
0,0,0,3763,400 CAPITAL MANAGEMENT LLC,ASSOCIATE QUANTITATIVE ANALYTICS
1,1,0,11079,400 CAPITAL MANAGEMENT LLC,INVESTMENT ANALYST
2,2,0,588,400 CAPITAL MANAGEMENT LLC,ANALYST FINANCE
3,3,1,9307,ABN AMRO CLEARING CHICAGO LLC,EXCHANGE FEE CONTROL ACCOUNTANT
4,4,1,19469,ABN AMRO CLEARING CHICAGO LLC,STOCK LOAN OPERATIONS ASSOCIATE
...,...,...,...,...,...
31680,31680,506,9043,THE HUNTINGTON NATIONAL BANK,ENGINEER
31681,31681,506,11962,THE HUNTINGTON NATIONAL BANK,LEAD ENGINEER
31682,31682,506,14776,THE HUNTINGTON NATIONAL BANK,QUANTITATIVE ANALYST
31683,31683,506,19732,THE HUNTINGTON NATIONAL BANK,SYSTEMS & APP TECHNOLOGIST


In [73]:
texts = texts.rename(columns={'EMPLOYER_NAME': 'COMPANY_NAME', 'JOB_TITLE': 'ROLE_NAME'})
texts

,COMPANY_NAME,ROLE_NAME,MINIMUM_EDUCATION,SPECIFIC_SKILLS
0,400 CAPITAL MANAGEMENT LLC,ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH,Master's,NaN
1,ABN AMRO CLEARING CHICAGO LLC,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST,Bachelor's,Bachelors degree or foreign equivalent plus te...
2,ACADIAN ASSET MANAGEMENT LLC,DEVOPS ENGINEER,Master's,"Continued from H. 10B. Similar experience, inc..."
3,ACADIAN ASSET MANAGEMENT LLC,SENIOR BUSINESS ANALYST,Bachelor's,Must have seven 7 years of experience in each ...
4,ACADIAN ASSET MANAGEMENT LLC,SENIOR SOFTWARE ENGINEER,Bachelor's,Must have five 5 years of experience in the fo...
...,...,...,...,...
5848,WELLS FARGO BANK NA,APPLICATION SYSTEMS ENGINEER 5,Bachelor's,MUST HAVE BACHELORS DEGREE IN COMPUTER SCIENCE...
5849,WELLS FARGO BANK NA,SOFTWARE ENGINEER SR.,Bachelor's,Requires a Bachelors degree in Electronic Engi...
5850,WILLIAM BLAIR COMPANY LLC,VICE PRESIDENT,Bachelor's,Education Requirement Bachelors degree or fore...
5851,WORLDQUANT LLC,ASSOCIATE QUANTITATIVE RESEARCH & OPTIMIZATION,Master's,"Background in education, training or experienc..."


In [74]:
texts = pd.merge(company_roles, texts, how='inner', on=['COMPANY_NAME','ROLE_NAME'])
texts

,COMPANY_ROLES_ID,COMPANY_ID,ROLE_ID,COMPANY_NAME,ROLE_NAME,MINIMUM_EDUCATION,SPECIFIC_SKILLS
0,14,1,20979,ABN AMRO CLEARING CHICAGO LLC,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST,Bachelor's,Bachelors degree or foreign equivalent plus te...
1,19,3,15110,AES US SERVICES LLC,QUANTITATIVE RISK ANALYST,Master's,Prior experience must include risk management ...
2,30,3,6488,AES US SERVICES LLC,CLOUD AND DATACENTER ARCHITECT,Bachelor's,Position requires Certified Solutions Architec...
3,34,3,13545,AES US SERVICES LLC,OPERATIONS ENGINEER,Master's,Job requires Master of Science degree in renew...
4,66,4,6103,AKUNA CAPITAL LLC,C++ DEVELOPER,Bachelor's,"Coursework, internship or research must have i..."
...,...,...,...,...,...,...,...
5920,31680,506,9043,THE HUNTINGTON NATIONAL BANK,ENGINEER,Bachelor's,NAbr br PLEASE NOTE The Huntington National Ba...
5921,31681,506,11962,THE HUNTINGTON NATIONAL BANK,LEAD ENGINEER,Bachelor's,NAbr br PLEASE NOTE The Huntington National Ba...
5922,31682,506,14776,THE HUNTINGTON NATIONAL BANK,QUANTITATIVE ANALYST,Master's,Telecommuting from any location within the Uni...
5923,31683,506,19732,THE HUNTINGTON NATIONAL BANK,SYSTEMS & APP TECHNOLOGIST,Bachelor's,NAbr br PLEASE NOTE The Huntington National Ba...


In [76]:
rk_list = []

for i1, r1 in texts.iterrows():
    string = ''
    if not pd.isna(r1['MINIMUM_EDUCATION']):
        string += r1['MINIMUM_EDUCATION'].upper().replace('\'', '') + ' '
    if not pd.isna(r1['SPECIFIC_SKILLS']):
        string += r1['SPECIFIC_SKILLS'].upper()
    if not string:
        continue
    
    for i2, r2 in skills.iterrows():
        if r2['SKILL_NAME'] in string:
            rk_list.append([r1['COMPANY_ROLES_ID'], r2['SKILL_ID']])

role_skills = pd.DataFrame(rk_list, columns=['COMPANY_ROLES_ID', 'SKILL_ID'])
role_skills

,COMPANY_ROLES_ID,SKILL_ID
0,14,57
1,14,107
2,19,27
3,19,37
4,19,82
...,...,...
45486,31681,107
45487,31682,37
45488,31683,107
45489,31684,32


In [77]:
role_skills = role_skills.drop_duplicates(ignore_index=True)
role_skills.index.name = 'COMPANY_ROLE_SKILLS_ID'
role_skills

,COMPANY_ROLES_ID,SKILL_ID
COMPANY_ROLE_SKILLS_ID,,
0,14,57
1,14,107
2,19,27
3,19,37
4,19,82
...,...,...
28639,31681,107
28640,31682,37
28641,31683,107


In [85]:
role_skills2 = pd.read_csv('company_role_skill.csv')

In [86]:
role_skills2 = role_skills2.append(role_skills, ignore_index=True)
role_skills2 = role_skills2.drop(['COMPANY_ROLE_SKILLS_ID'], 1)
role_skills2 = role_skills2.drop_duplicates(ignore_index=True)
role_skills2

<ipython-input-86-494fa282d291>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  role_skills2 = role_skills2.drop(['COMPANY_ROLE_SKILLS_ID'], 1)


,COMPANY_ROLES_ID,SKILL_ID
0,2,24
1,12,31
2,39,30
3,49,30
4,52,88
...,...,...
32590,31681,107
32591,31682,37
32592,31683,107
32593,31684,32


In [87]:
role_skills2.index.name = 'COMPANY_ROLE_SKILLS_ID'
role_skills2

,COMPANY_ROLES_ID,SKILL_ID
COMPANY_ROLE_SKILLS_ID,,
0,2,24
1,12,31
2,39,30
3,49,30
4,52,88
...,...,...
32590,31681,107
32591,31682,37
32592,31683,107


In [90]:
role_skills2.to_csv('company_role_skills.csv')